<a href="https://colab.research.google.com/github/HanbinKim1004/Crop_Diseases_Diagnosis/blob/main/BangHanBae_full_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###경로 설정 & 라이브러리 import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = 'dm_nfnet_f0'
opt_name = 'RAdam'

In [ ]:
model_save_path =  '/content/drive/MyDrive/한빈남용/plan_B/data/'
model_save_name = 'kfold_' + model_name + '_' + opt_name + '.pt'

In [ ]:
from google.colab import output

!mkdir /content/data
%cd /content/data

! unzip "/content/drive/MyDrive/한빈남용/train.zip"
! unzip "/content/drive/MyDrive/한빈남용/test.zip"

output.clear()

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 11.9 MB/s 


In [ ]:
import warnings; warnings.filterwarnings('ignore')
import random
import json
import cv2
import time
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

from skimage.transform import warp, rotate, AffineTransform, ProjectiveTransform
from skimage.util import random_noise
from scipy import ndimage

import timm
from timm.data.auto_augment import auto_augment_transform
from timm.data.auto_augment import rand_augment_transform
from timm.data.transforms import RandomResizedCropAndInterpolation
from PIL import Image
sys.getdefaultencoding()

'utf-8'

In [ ]:
train_csv = sorted(glob('train/*/*.csv'))
train_jpg = sorted(glob('train/*/*.jpg'))
train_json = sorted(glob('train/*/*.json'))

###데이터 전처리 & 로드

- 환경 데이터 전처리

In [ ]:
max_len = 600
csv_features = ['내부 온도 1 평균', '내부 습도 1 평균', '내부 이슬점 평균']
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
first_env = pd.read_csv(train_csv[0])[csv_features]
first_env = first_env.replace('-',np.nan)
first_env = pd.DataFrame(imputer.fit_transform(first_env))
env_concat = first_env.astype(float).diff().iloc[1:].values
for csv_path in tqdm(train_csv[1:]):
    tmp = pd.read_csv(csv_path)[csv_features]
    tmp = tmp.replace('-', np.nan)
    tmp = pd.DataFrame(imputer.fit_transform(tmp))
    tmp = tmp.astype(float).diff().replace(np.nan, 0.).values
    if len(tmp) <= 1:
        continue
    env_concat = np.vstack((env_concat, tmp[1:]))
print('\n(num_row, num_feat): ', env_concat.shape, sep='')

100%|██████████| 5766/5766 [01:39<00:00, 57.72it/s]


(num_row, num_feat): (1772360, 3)


In [ ]:
scaler = RobustScaler().fit(env_concat)

In [ ]:
def prepro_csv(csv_file, max_len=200):
    df = pd.read_csv(csv_file)[csv_features]
    df = df.replace('-', np.nan).dropna()
    df = pd.DataFrame(df).astype(float).diff()
    df = df.replace(np.nan, 0.)
    scaled = scaler.transform(df)
    scaled_df = pd.DataFrame(scaled)
    pad = np.zeros((max_len, len(scaled_df.columns)))
    length = min(max_len, len(scaled_df))
    pad[-length:] = scaled_df.to_numpy()[-length:]
    env_output = np.round(pad.T, 6)
    return env_output

- 레이블 데이터 전처리

In [ ]:
crops = []
diseases = []
risks = []
labels_str = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"  
        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels_str.append(label)
label2int = sorted(np.unique(labels_str))
label2int = {key:value for key,value in zip(label2int, range(len(label2int)))}
labels = [label2int[k] for k in labels_str]
int2label = {}
for key, value in label2int .items():
    int2label[value] = key
len(labels)

5767

- 데이터 로드

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    return img

In [ ]:
imgs = []
for i in tqdm(train_jpg):
    imgs.append(img_load(i))

100%|██████████| 5767/5767 [00:28<00:00, 202.64it/s]


In [ ]:
envs = []
for i in tqdm(train_csv):
    envs.append(prepro_csv(i, max_len))

100%|██████████| 5767/5767 [00:54<00:00, 104.96it/s]


In [ ]:
bbxs = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        tmp = json.load(f)
    X = int(tmp['annotations']['bbox'][0]['x'])
    H = int(tmp['annotations']['bbox'][0]['h'])
    Y = int(tmp['annotations']['bbox'][0]['y'])
    W = int(tmp['annotations']['bbox'][0]['w'])
    bbxs.append([X, X+W, Y, Y+H])

In [ ]:
part = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        tmp = json.load(f)
    num_part = len(tmp['annotations']['part'])
    if num_part == 0:
        part.append([])
    else:
        sub_list = []
        for i in range(num_part):
            X = int(tmp['annotations']['part'][i]['x'])
            H = int(tmp['annotations']['part'][i]['h'])
            Y = int(tmp['annotations']['part'][i]['y'])
            W = int(tmp['annotations']['part'][i]['w'])
            sub_list.append([X, X+W, Y, Y+H])
        part.append(sub_list)

- 데이터 오버샘플링

In [ ]:
freq = {i:labels.count(i) for i in set(labels)}
freqdf = pd.DataFrame(freq.values(), columns=['Label frequency']); freqdf

,Label frequency
0,810
1,143
2,189
3,1177
4,154
5,111
6,42
7,166
8,142
9,156


In [ ]:
imbalance_label_1 = np.unique([i for i in labels if freq[i] < 20])
imbalance_label_2 = np.unique([i for i in labels if freq[i] >= 20 and freq[i] <= 42])
imbalance_label_3 = np.unique([i for i in labels if freq[i] > 42 and freq[i] < 70])
print(imbalance_label_1, '\n', imbalance_label_2, '\n', imbalance_label_3, sep='')

[19 20 22 23]
[ 6 18 21 24]
[12]


In [ ]:
oversample_num_1 = 12
oversample_num_2 = 5
oversample_num_3 = 2

In [ ]:
imgs_oversampled = imgs.copy()
envs_oversampled = envs.copy()
labels_oversampled = labels.copy()
bbxs_oversampled = bbxs.copy()
part_oversampled = part.copy()

In [ ]:
for idx, label in enumerate(tqdm(labels)):
    if label in imbalance_label_1:
        for i in range(oversample_num_1):
            imgs_oversampled.append(imgs[idx])
            envs_oversampled.append(envs[idx])
            labels_oversampled.append(label)
            bbxs_oversampled.append(bbxs[idx])
            part_oversampled.append(part[idx])
    elif label in imbalance_label_2:
        for i in range(oversample_num_2):
            imgs_oversampled.append(imgs[idx])
            envs_oversampled.append(envs[idx])
            labels_oversampled.append(label)
            bbxs_oversampled.append(bbxs[idx])
            part_oversampled.append(part[idx])
    elif label in imbalance_label_3:
        for i in range(oversample_num_3):
            imgs_oversampled.append(imgs[idx])
            envs_oversampled.append(envs[idx])
            labels_oversampled.append(label)
            bbxs_oversampled.append(bbxs[idx])
            part_oversampled.append(part[idx])
    else:
        continue

100%|██████████| 5767/5767 [00:00<00:00, 98593.97it/s]


In [ ]:
freq_oversampled = {i:labels_oversampled.count(i) for i in set(labels_oversampled)}
freqdf_oversampled = pd.DataFrame(freq_oversampled.values(), columns=['Label frequency']); freqdf_oversampled

,Label frequency
0,810
1,143
2,189
3,1177
4,154
5,111
6,252
7,166
8,142
9,156


### 데이터 증강 함수

In [ ]:
img_size = (512, 512)

In [ ]:
def randRange(a, b):
    return np.round(np.random.rand() * (b - a) + a,  2)
def rotation(img): 
    rotated_img = rotate(img, angle=random.choice([-15, 15, 30, -30, 90, -90]))
    norm_image = cv2.normalize(rotated_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    rotated_img = norm_image.astype(np.uint8)
    return rotated_img
def up_down(img):
    return np.flipud(img)
def left_right(img):
    return np.fliplr(img)
def noise(img):
    noised_img = random_noise(img, var=random.choice([3e-2, 3e-3]))
    norm_image = cv2.normalize(noised_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    noised_img = norm_image.astype(np.uint8)
    return noised_img
def brightness(img):
    return img + random.choice([20, 30, 40])
def contrast(img):
    contrasted_img = img * random.choice([5e-3, 6e-3])
    norm_image = cv2.normalize(contrasted_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    contrasted_img = norm_image.astype(np.uint8)
    return contrasted_img
def blur(img):
    return ndimage.uniform_filter(img, size=(random.choice([4, 6]), random.choice([4, 6]), random.choice([1, 1.3])))
def random_affine(img):
    tform = AffineTransform(scale=(randRange(0.75, 1.3), randRange(0.75, 1.3)),
                                                   rotation=randRange(-0.25, 0.25),
                                                   shear=randRange(-0.2, 0.2),
                                                   translation=(randRange(-img.shape[0]//10, img.shape[0]//10), 
                                                   randRange(-img.shape[1]//10, img.shape[1]//10)))
    affined_img = np.float32(warp(img, tform.inverse, mode='reflect'))
    norm_image = cv2.normalize(affined_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    affined_img = norm_image.astype(np.uint8)
    return affined_img
def random_perspective(img):
    region = 1/4
    A = np.array([[0, 0], [0, img.shape[0]], [img.shape[1], img.shape[0]], [img.shape[1], 0]])
    B = np.array([[int(randRange(0, img.shape[1] * region)), int(randRange(0, img.shape[0] * region))], 
                            [int(randRange(0, img.shape[1] * region)), int(randRange(img.shape[0] * (1-region), img.shape[0]))], 
                            [int(randRange(img.shape[1] * (1-region), img.shape[1])), int(randRange(img.shape[0] * (1-region), img.shape[0]))], 
                            [int(randRange(img.shape[1] * (1-region), img.shape[1])), int(randRange(0, img.shape[0] * region))], 
                            ])
    pt = ProjectiveTransform()
    pt.estimate(A, B)
    
    perspectived_img = np.float32(warp(img, pt, output_shape=img.shape[:2]))
    norm_image = cv2.normalize(perspectived_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    perspectived_img = norm_image.astype(np.uint8)
    return perspectived_img

In [ ]:
aug_func_list = [rotation, up_down, left_right, noise, brightness, contrast, blur, random_affine, random_perspective]
def series_aug(img, aug_func_list):
    num_func = random.choice(list(range(2, 7)))
    chosen_func = random.sample(aug_func_list, num_func)
    random.shuffle(chosen_func)
    for func in chosen_func:
        img = func(img)
    return img

In [ ]:
def skimg_aug(img):
    img_augmented = series_aug(img, aug_func_list)
    img_augmented_resized = cv2.resize(img_augmented, img_size)
    return img_augmented_resized

In [ ]:
autoaug = auto_augment_transform(config_str='original', hparams={'translate_const': 100, 'img_mean': (124, 116, 104)})
randaug = rand_augment_transform(config_str='rand-m9-mstd0.5', hparams={'translate_const': 117, 'img_mean': (124, 116, 104)})
resizecrop = RandomResizedCropAndInterpolation(size=random.choice([300, 400]))
timm_func_list = [autoaug, randaug, resizecrop]
def timm_aug(img):
    img = Image.fromarray(np.uint8(img))
    chosen_func = random.sample(timm_func_list, random.choice([1, 2, 3]))
    random.shuffle(chosen_func)
    for func in chosen_func:
        img = func(img)
    img_augmented_resized = cv2.resize(np.array(img), img_size)
    return img_augmented_resized

In [ ]:
def mixup_aug(idx, beta):
    sampled_idx = random.sample(list(train_idx), 1)[0]
    img1 = cv2.resize(imgs_oversampled[idx], img_size)
    img2 =  cv2.resize(imgs_oversampled[sampled_idx], img_size)
    label1 = labels_oversampled[idx]
    label2 = labels_oversampled[sampled_idx]
    alpha= np.random.beta(beta, beta)
    mixup_img = alpha * img1 /255 + (1-alpha) *  img2 /255
    mixup_label = alpha * label1 + (1-alpha) * label2
    norm_image = cv2.normalize(mixup_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    mixup_img = norm_image.astype(np.uint8)
    return mixup_img, np.round(mixup_label, 4)

In [ ]:
pre_torch_transformer = transforms.Compose([transforms.ToPILImage(),
                                                                                         transforms.RandomPosterize(bits=3, p=0.25),
                                                                                         transforms.ColorJitter(brightness=randRange(0.15, 0.3), contrast=randRange(0.15, 0.3), saturation=randRange(0.15, 0.3), hue=randRange(0.15, 0.3)), # 밝기, 대비, 채도, 색조
                                                                                         transforms.RandomInvert(p=0.1),
                                                                                         transforms.RandomAdjustSharpness(random.choice([2, 4]), p=0.75)
                                                                                         ])
post_torch_transformer = transforms.Compose([transforms.ToPILImage(),
                                                                                         transforms.RandAugment(num_ops=random.randint(1, 4), magnitude=random.randint(1, 4))
                                                                                         ])

In [ ]:
def torch_aug_with_bbx(img, idx):
    torch_augmented = np.array(pre_torch_transformer(img))
    x1, x2, y1, y2 = bbxs_oversampled[idx]
    mix_type = random.randint(0, 1)
    post_aug = random.randint(0, 1)
    if mix_type == 0:
        torch_augmented[y1:y2,x1:x2] = img[y1:y2,x1:x2]
        if post_aug == 1:
            torch_augmented = np.array(post_torch_transformer(torch_augmented))
            return cv2.resize(torch_augmented, img_size)
        elif post_aug == 0:
            return cv2.resize(torch_augmented, img_size)
    elif mix_type == 1:
        img[y1:y2,x1:x2] = torch_augmented[y1:y2,x1:x2]
        if post_aug == 1:
            img = np.array(post_torch_transformer(img))
            return cv2.resize(img, img_size)
        elif post_aug == 0:
            return cv2.resize(img, img_size)

In [ ]:
def torch_aug_with_part_bbx(img, idx):
    torch_augmented = np.array(pre_torch_transformer(img))
    part_list = part_oversampled[idx]
    mix_type = random.randint(0, 1)
    post_aug = random.randint(0, 1)
    if len(part_list) == 0:
        if post_aug == 1:
            torch_augmented = np.array(post_torch_transformer(torch_augmented))
            return cv2.resize(torch_augmented, img_size)
        elif post_aug == 0:
            return cv2.resize(torch_augmented, img_size)
    else:
        if mix_type == 0:
            for i in range(len(part_list)):
                x1, x2, y1, y2 = part_list[i]
                torch_augmented[y1:y2,x1:x2] = img[y1:y2,x1:x2]
            if post_aug == 1:
                torch_augmented = np.array(post_torch_transformer(torch_augmented))
                return cv2.resize(torch_augmented, img_size)
            elif post_aug == 0:
                return cv2.resize(torch_augmented, img_size)
        elif mix_type == 1:
            for i in range(len(part_list)):
                x1, x2, y1, y2 = part_list[i]
                img[y1:y2,x1:x2] = torch_augmented[y1:y2,x1:x2]
            if post_aug == 1:
                img = np.array(post_torch_transformer(img))
                return cv2.resize(torch_augmented, img_size)
            elif post_aug == 0:
                return cv2.resize(img, img_size)

In [ ]:
def edge_aug(img):
    edge_type = random.randint(0, 2)
    post_aug = random.randint(0, 1)
    if edge_type == 0:
        edge = cv2.Canny(img, random.choice([60, 80]), random.choice([60, 80]))
        edge = np.stack((edge,)*3, axis=-1)
    elif edge_type == 1:
        edge = cv2.Sobel(img, -1, 0, 1)
    elif edge_type == 2:
        edge = cv2.Laplacian(img, -1)
    img_edge_added = edge + img
    if post_aug == 1:
        img_augmented = timm_aug(img_edge_added)
    else:
        img_augmented = img_edge_added
    img_augmented_resized = cv2.resize(img_augmented, img_size)
    return img_augmented_resized

In [ ]:
def resize_only(img):
    img_resized = cv2.resize(img, img_size)
    return img_resized

In [ ]:
def scaling(x, sigma=0.05):
    factor = np.random.normal(loc=1., scale=sigma, size=x.shape)
    return np.multiply(x, factor)
def temporal_aug(envs):
    aug = np.round(np.random.rand(), 2)
    if aug >= 0.4:
        df = pd.DataFrame(envs)
        df_nonzero = df.loc[:, (df != 0).any(axis=0)]
        df_nonzero_aug = scaling(df_nonzero)
        df[df_nonzero_aug.columns] = df_nonzero_aug
        return df.values
    else:
        return envs

### 데이터셋 정의

In [ ]:
class custom_dataset(Dataset):
    def __init__(self, files, envs, labels, mode='train'):
        self.mode = mode
        self.files = files
        self.envs = envs
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        files = self.files[i]

        if self.mode=='train':
            aug_type = random.randint(0, 6)
            if aug_type in list(range(0, 1)):
                files, label = mixup_aug(i, beta=1)
            elif aug_type in list(range(1, 2)):
                files = timm_aug(files)
                label = self.labels[i]
            elif aug_type in list(range(2, 3)):
                files = skimg_aug(files)
                label = self.labels[i]
            elif aug_type in list(range(3, 4)):
                files = torch_aug_with_bbx(files, i)
                label = self.labels[i]
            elif aug_type in list(range(4, 5)):
                files = torch_aug_with_part_bbx(files, i)
                label = self.labels[i]
            elif aug_type in list(range(5, 6)):
                files = edge_aug(files)
                label = self.labels[i]
            else:
                files = resize_only(files)
                label = self.labels[i]

            files = transforms.ToTensor()(files)
            envs = temporal_aug(self.envs[i])
            return files, envs, label

        elif self.mode=='valid':
            files = resize_only(files)
            files = transforms.ToTensor()(files)
            envs = self.envs[i]
            label = self.labels[i]
            return files, envs, label

        elif self.mode=='test':
            files = resize_only(files)
            files = transforms.ToTensor()(files)
            envs = self.envs[i]
            return files, envs

### 모델 정의

In [ ]:
num_class = len(np.unique(labels))
rnn_hidden_dim = 512
dropout_rate = 0.4
cnn_output_dim = 1000
rnn_output_dim = 128
env_temporal_len = max_len
env_feature_len = len(csv_features)

In [ ]:
class CNN_Encoder(nn.Module):
    def __init__(self):
        super(CNN_Encoder, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

In [ ]:
class RNN_Decoder(nn.Module):
    def __init__(self, env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate):
        super(RNN_Decoder, self).__init__()
        self.lstm = nn.LSTM(env_temporal_len, rnn_hidden_dim)
        self.rnn_fc = nn.Linear(env_feature_len * rnn_hidden_dim, rnn_output_dim)
        self.final_layer = nn.Linear(cnn_output_dim + rnn_output_dim, num_class)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, enc_out, dec_inp):
        hidden, _ = self.lstm(dec_inp)
        hidden = hidden.view(hidden.size(0), -1)
        hidden = self.rnn_fc(hidden)
        concat = torch.cat([enc_out, hidden], dim=1)
        fc_input = concat
        output = self.dropout((self.final_layer(fc_input)))
        return output

In [ ]:
class CNN2RNN(nn.Module):
    def __init__(self, env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate):
        super(CNN2RNN, self).__init__()
        self.cnn = CNN_Encoder()
        self.rnn = RNN_Decoder(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate)

    def forward(self, img, seq):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq)
        return output

In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda'); device

device(type='cuda')

In [ ]:
learning_rate = 4e-5
label_smoothing = 0.95

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
gscaler = torch.cuda.amp.GradScaler()

### 데이터 로더

In [ ]:
def accuracy_function(real, pred):
    score = f1_score(real, pred, average='macro')
    return score

In [ ]:
n_splits = 5
batch_size = 32
num_epochs = 30
best_models = []
k_fold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=41)
for fold, (train_idx, valid_idx) in enumerate(k_fold.split(imgs_oversampled, labels_oversampled)):
  print(f'[fold: {fold}]')
  
  torch.cuda.empty_cache()

  train_imgs = list(map(imgs_oversampled.__getitem__, train_idx))
  train_envs = list(map(envs_oversampled.__getitem__, train_idx))
  train_labels = list(map(labels_oversampled.__getitem__, train_idx))
  val_imgs = list(map(imgs_oversampled.__getitem__, valid_idx))
  val_envs = list(map(envs_oversampled.__getitem__, valid_idx))
  val_labels = list(map(labels_oversampled.__getitem__, valid_idx))
  
  train_dataset = custom_dataset(train_imgs, train_envs, train_labels, mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True, num_workers=8)

  valid_dataset = custom_dataset(val_imgs, val_envs, val_labels, mode='valid')
  valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size, pin_memory=True, num_workers=8)

  model = CNN2RNN(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate).to(device)
  optimizer = torch.optim.RAdam(model.parameters(), lr=learning_rate)

  best_valid_f1 = 0
  for epoch in tqdm(range(num_epochs)):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in tqdm(train_loader):
      optimizer.zero_grad()
      img = torch.tensor(batch[0] , dtype=torch.float32, device=device)
      env = torch.tensor(batch[1], dtype=torch.float32, device=device)
      y = torch.tensor(batch[2], dtype=torch.long, device=device)
      with torch.cuda.amp.autocast():
        pred = model(img, env)
      loss = criterion(pred, y)

      gscaler.scale(loss).backward()
      gscaler.step(optimizer)
      gscaler.update()
        
      train_loss += loss.item()/len(train_loader)
      train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
      train_y += y.detach().cpu().numpy().tolist()
    train_f1 = accuracy_function(train_y, train_pred)
    
    model.eval()
    valid_loss = 0
    valid_pred=[]
    valid_y=[]
    with torch.no_grad():
      for batch in tqdm(valid_loader):
        img = torch.tensor(batch[0], dtype=torch.float32, device=device)
        env = torch.tensor(batch[1], dtype=torch.float32, device=device)
        y = torch.tensor(batch[2], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
          pred = model(img, env)
        loss = criterion(pred, y)
        valid_loss += loss.item()/len(valid_loader)
        valid_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        valid_y += y.detach().cpu().numpy().tolist()
      valid_f1 = accuracy_function(valid_y, valid_pred)
    if valid_f1 >= best_valid_f1:
        best_valid_f1 = valid_f1
        best_model = model
        torch.save(model.state_dict(), model_save_path + "fold" + str(fold) + "_" + model_save_name)
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{num_epochs}              Train  time : {TIME:.0f}s/{TIME*(num_epochs-epoch-1):.0f}s')
    print(f'TRAIN loss : {train_loss:.5f}    f1: {train_f1:.5f}')
    print(f'VALID loss : {valid_loss:.5f}    f1: {valid_f1:.5f}    best: {best_valid_f1:.5f}')
    print()
  
  best_models.append(best_model)
print(best_models)

[fold: 0]


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-dnf-weights/dm_nfnet_f0-604f9c3a.pth" to /root/.cache/torch/hub/checkpoints/dm_nfnet_f0-604f9c3a.pth
100%|██████████| 182/182 [08:26<00:00,  2.79s/it]

  3%|▎         | 1/30 [09:00<4:21:11, 540.40s/it]

epoch : 1/30              Train  time : 540s/15671s
TRAIN loss : 3.24095    f1: 0.06684
VALID loss : 3.21561    f1: 0.15151    best: 0.15151




100%|██████████| 182/182 [08:24<00:00,  2.77s/it]

  7%|▋         | 2/30 [17:57<4:11:20, 538.60s/it]

epoch : 2/30              Train  time : 537s/15046s
TRAIN loss : 3.21513    f1: 0.19171
VALID loss : 3.20971    f1: 0.46156    best: 0.46156



...

epoch : 28/30              Train  time : 439s/878s
TRAIN loss : 3.20795    f1: 0.52926
VALID loss : 3.20206    f1: 0.95967    best: 0.96731




100%|██████████| 182/182 [06:47<00:00,  2.24s/it]

 97%|█████████▋| 29/30 [3:32:31<07:19, 439.33s/it]

epoch : 29/30              Train  time : 440s/440s
TRAIN loss : 3.20824    f1: 0.51626
VALID loss : 3.20296    f1: 0.96778    best: 0.96778




100%|██████████| 182/182 [06:47<00:00,  2.24s/it]

100%|██████████| 30/30 [3:39:50<00:00, 439.70s/it]

epoch : 30/30              Train  time : 440s/0s
TRAIN loss : 3.20789    f1: 0.52682
VALID loss : 3.20257    f1: 0.96986    best: 0.96986

[CNN2RNN(
  (cnn): CNN_Encoder(
    (model): NormFreeNet(
      (stem): Sequential(
        (conv1): ScaledStdConv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2))
        (act2): GammaAct()
        (conv2): ScaledStdConv2dSame(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act3): GammaAct()
        (conv3): ScaledStdConv2dSame(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act4): GammaAct()
        (conv4): ScaledStdConv2dSame(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2dSame(128, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): GammaAct()
            (conv1): ScaledStdConv2dSame(128, 1

train 끝!!!

In [ ]:
! unzip "/content/drive/MyDrive/한빈남용/test.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: test/65823/65823.csv    
  inflating: test/65823/65823.jpg    
   creating: test/65824/
  inflating: test/65824/65824.jpg    
  inflating: test/65824/65824.csv    
   creating: test/65825/
  inflating: test/65825/65825.csv    
  inflating: test/65825/65825.jpg    
   creating: test/65827/
  inflating: test/65827/65827.jpg    
  inflating: test/65827/65827.csv    
   creating: test/65828/
  inflating: test/65828/65828.jpg    
  inflating: test/65828/65828.csv    
   creating: test/65829/
  inflating: test/65829/65829.jpg    
  inflating: test/65829/65829.csv    
   creating: test/65830/
  inflating: test/65830/65830.jpg    
  inflating: test/65830/65830.csv    
   creating: test/65831/
  inflating: test/65831/65831.jpg    
  inflating: test/65831/65831.csv    
   creating: test/65832/
  inflating: test/65832/65832.jpg    
  inflating: test/65832/65832.csv    
   creating: test/65833/
  inflating: test/65833/65833.jpg    
  inflating: test

In [ ]:
test_csv = sorted(glob('test/*/*.csv'))
test_jpg = sorted(glob('test/*/*.jpg'))

In [ ]:
test_imgs = []
for i in tqdm(test_jpg):
    test_imgs.append(img_load(i))

100%|██████████| 51906/51906 [04:40<00:00, 185.23it/s]


In [ ]:
test_envs = []
for i in tqdm(test_csv):
    test_envs.append(prepro_csv(i, max_len))

100%|██████████| 51906/51906 [09:41<00:00, 89.30it/s]


In [ ]:
class test_dataset(Dataset):
    def __init__(self, files, envs, labels):
        self.files = files
        self.envs = envs
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        files = self.files[i]
        files = resize_only(files)
        files = transforms.ToTensor()(files)
        envs = self.envs[i]
        return files, envs

In [ ]:
batch_size = 32

In [ ]:
test_dataset = test_dataset(test_imgs, test_envs, labels)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, pin_memory=True, num_workers=8)

In [ ]:
model1_path = glob(model_save_path + 'fold0_kfold_dm_nfnet_f0_AdamW.pt')[-1]
model2_path = glob(model_save_path + 'fold1_kfold_dm_nfnet_f0_AdamW.pt')[-1]
model3_path = glob(model_save_path + 'fold2_kfold_dm_nfnet_f0_AdamW.pt')[-1]
model4_path = glob(model_save_path + 'fold3_kfold_dm_nfnet_f0_AdamW.pt')[-1]
model5_path = glob(model_save_path + 'fold4_kfold_dm_nfnet_f0_AdamW.pt')[-1]

In [ ]:
model1 = CNN2RNN(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate)
model1.load_state_dict(torch.load(model1_path))
model1.to(device)
model1.eval()

CNN2RNN(
  (cnn): CNN_Encoder(
    (model): NormFreeNet(
      (stem): Sequential(
        (conv1): ScaledStdConv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2))
        (act2): GammaAct()
        (conv2): ScaledStdConv2dSame(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act3): GammaAct()
        (conv3): ScaledStdConv2dSame(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act4): GammaAct()
        (conv4): ScaledStdConv2dSame(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2dSame(128, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): GammaAct()
            (conv1): ScaledStdConv2dSame(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act2): GammaAct()
            (conv2): ScaledStdConv2dSame(128, 128, kernel_size=(3, 3),

In [ ]:
model2 = CNN2RNN(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate)
model2.load_state_dict(torch.load(model2_path))
model2.to(device)
model2.eval()

CNN2RNN(
  (cnn): CNN_Encoder(
    (model): NormFreeNet(
      (stem): Sequential(
        (conv1): ScaledStdConv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2))
        (act2): GammaAct()
        (conv2): ScaledStdConv2dSame(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act3): GammaAct()
        (conv3): ScaledStdConv2dSame(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act4): GammaAct()
        (conv4): ScaledStdConv2dSame(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2dSame(128, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): GammaAct()
            (conv1): ScaledStdConv2dSame(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act2): GammaAct()
            (conv2): ScaledStdConv2dSame(128, 128, kernel_size=(3, 3),

In [ ]:
model3 = CNN2RNN(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate)
model3.load_state_dict(torch.load(model3_path))
model3.to(device)
model3.eval()

CNN2RNN(
  (cnn): CNN_Encoder(
    (model): NormFreeNet(
      (stem): Sequential(
        (conv1): ScaledStdConv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2))
        (act2): GammaAct()
        (conv2): ScaledStdConv2dSame(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act3): GammaAct()
        (conv3): ScaledStdConv2dSame(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act4): GammaAct()
        (conv4): ScaledStdConv2dSame(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2dSame(128, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): GammaAct()
            (conv1): ScaledStdConv2dSame(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act2): GammaAct()
            (conv2): ScaledStdConv2dSame(128, 128, kernel_size=(3, 3),

In [ ]:
model4 = CNN2RNN(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate)
model4.load_state_dict(torch.load(model4_path))
model4.to(device)
model4.eval()

CNN2RNN(
  (cnn): CNN_Encoder(
    (model): NormFreeNet(
      (stem): Sequential(
        (conv1): ScaledStdConv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2))
        (act2): GammaAct()
        (conv2): ScaledStdConv2dSame(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act3): GammaAct()
        (conv3): ScaledStdConv2dSame(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act4): GammaAct()
        (conv4): ScaledStdConv2dSame(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2dSame(128, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): GammaAct()
            (conv1): ScaledStdConv2dSame(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act2): GammaAct()
            (conv2): ScaledStdConv2dSame(128, 128, kernel_size=(3, 3),

In [ ]:
model5 = CNN2RNN(env_temporal_len, rnn_hidden_dim, rnn_output_dim, env_feature_len, num_class, dropout_rate)
model5.load_state_dict(torch.load(model5_path))
model5.to(device)
model5.eval()

CNN2RNN(
  (cnn): CNN_Encoder(
    (model): NormFreeNet(
      (stem): Sequential(
        (conv1): ScaledStdConv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2))
        (act2): GammaAct()
        (conv2): ScaledStdConv2dSame(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act3): GammaAct()
        (conv3): ScaledStdConv2dSame(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act4): GammaAct()
        (conv4): ScaledStdConv2dSame(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2dSame(128, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): GammaAct()
            (conv1): ScaledStdConv2dSame(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act2): GammaAct()
            (conv2): ScaledStdConv2dSame(128, 128, kernel_size=(3, 3),

In [ ]:
with torch.no_grad():

  preds = []
  for batch in tqdm(test_loader):
    img = torch.tensor(batch[0], dtype=torch.float32, device=device)
    env = torch.tensor(batch[1], dtype=torch.float32, device=device)
    output1 = nn.Softmax(dim=-1)(model1(img, env))
    output2 = nn.Softmax(dim=-1)(model2(img, env))
    output3 = nn.Softmax(dim=-1)(model3(img, env))
    output4 = nn.Softmax(dim=-1)(model4(img, env))
    output5 = nn.Softmax(dim=-1)(model5(img, env))

    output = output1 + output2 + output3 + output4 + output5
    pred = torch.argmax(output, dim=1).tolist()
    preds.extend(pred)
test_preds_converted = [str(int2label[i]) for i in tqdm(preds)]
print('\n', len(test_preds_converted), sep='')

100%|██████████| 51906/51906 [00:00<00:00, 1314425.10it/s]


51906


In [ ]:
submission_template = pd.read_csv('/content/drive/MyDrive/한빈남용/plan_B/data/sample_submission.csv')
submission_template.head()

,image,label
0,10000,0_00_0
1,10001,0_00_0
2,10002,0_00_0
3,10003,0_00_0
4,10004,0_00_0


In [ ]:
submission = submission_template.copy()
submission['label'] = test_preds_converted
print(len(submission))
submission.head()

51906


,image,label
0,10000,6_00_0
1,10001,5_b6_1
2,10002,4_00_0
3,10003,3_00_0
4,10004,3_b8_1


In [ ]:
submission_file_name = 'submission_Kfold_dm_nfnet_f0_AdamW.csv'
submission.to_csv(model_save_path + submission_file_name, index=False)

Test 끝!!!